In [25]:
#import audio
import librosa

In [2]:
samples, sr = librosa.load('../tests/silence_10s.mp3')

In [3]:
import random
import numpy as np

# Chunk extraction

### Helper function: `wraparound_extract()`

In [4]:
def wraparound_extract(original, begin, length):
    '''
    Extracts elements from numpy.array in a "wraparound" fashion
    
    Extracts a certain number of elements from 
    a numpy.array starting at a certain position.
    If the chosen position and length go
    past the end of the array, the extraction
    "wraps around" to the beginning of the numpy.array
    as many times as necessary. For instance:
    
    wraparound_extract(
        original = [0, 5, 10],
        begin = 1, 
        length = 7) -> [5, 10, 0, 5, 10, 0, 5]
    
    Args:
        original (np.array): the original array 
        begin (int): beginning position to extract
        length (int): number of elements to extract
    '''

    # Get `head`: the array after the beginning position
    assert(type(original) == np.ndarray)
    len_original = original.shape[0]
    begin = begin % len_original
    head = original[begin:]
    len_head = head.shape[0]

    # Number of elements we require for full wrap-around
    wrap_needed = length - len_head

    # Generate the desired list, wrapped if necessary
    if wrap_needed > 0:
        repeats = np.tile(original, int(wrap_needed/len_original))
        tail = np.array(original[ : (wrap_needed % len_original)])
        desired_list = np.concatenate((head, repeats, tail))
    else:
        desired_list = original[begin:begin+length]
    
    #print(desired_list)
    return desired_list

### Tests of `wraparound_extract()`

In [5]:
import numpy.testing as npt

# test zero beginning, not getting to end of original array
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 0, length = 1), np.array([0]))

# test zero beginning, not getting to end of original array
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 0, length = 2), np.array([0, 1]))

# test zero beginning, not wrapping
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 0, length = 2), np.array([0, 1]))

# test zero beginning, wrapping around
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 0, length = 3), np.array([0, 1, 0]))

# test nonzero beginning, not wrapping
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 1, length = 1), np.array([1]))

# test nonzero beginning, wrapping around
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 1, length = 3), np.array([1, 0, 1]))

# test multiwrap
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 1, length = 10), np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0]))

# test wrapping around beginning
npt.assert_array_equal(wraparound_extract(original = np.array([0, 1]), begin = 5, length = 3), np.array([1, 0, 1]))



### Main function: `get_chunk()`

In [6]:
def get_chunk(
    samples, 
    sample_rate,
    start_position = None, # randomize start position
    duration = 5, # 5 seconds
    duration_jitter = 0.5, #jitter duration +- 0.5s
    chance_random_skip = 0.3 #randomly skip 30% of the time
):
    '''
    Extracts chunk of audio with some augmentation
    
    Extracts samples of audio from a master list
    of samples. 
    
    Available data augmentation options include:
        - selecting a position to start extracting from
          or allowing function to randomly choose start
        - selecting duration of chunk and allowing
          for random jitter of duration
        - randomly skipping some number of samples from
          0 to the length of the chunk
    
    If the chunk to be extracted reaches the end of the
    samples, the chunk will "wrap around" and start
    reading from the beginning of the samples.
    
    Args
        samples (numpy.array) - audio samples loaded
            by librosa.load or audio.load
        sample_rate (int) - sample rate of `samples`
        start_position (int) - position in the file to start
            extracting samples from. If None, the start position 
            is chosen randomly
        duration (float) - desired duration, in seconds, 
            of chunk to extract
        duration_jitter (float) - if this value is not 0,
            the duration of the chunk extracted will 
            be randomly selected from the range 
            (duration - duration_jitter, duration + duration_jitter)
        chance_random_skip (float between 0 and 1) - 
            percent chance of random skipping. In a random skip,
            a position within the chunk will be randomly
            selected, and from that position in the 
            audio file, a random number of samples will 
            be skipped. The number of samples skipped is between
            0 and the number of samples in the entire chunk
    
    Returns
        samples
    '''
    
    # Get a random start position
    num_samples = len(samples)
    if not start_position:
        start_position = random.randint(0, num_samples)

    # Convert seconds to samples
    seconds_to_extract = duration + random.uniform(-duration_jitter, duration_jitter)
    samples_to_extract = int(seconds_to_extract * sample_rate)
    
    # Get chunks with skip in the middle with probability = chance_random_skip
    if random.random() < chance_random_skip:
        position_to_skip = random.randint(0, samples_to_extract)
        amount_to_skip = random.randint(0, samples_to_extract)

        chunk_1_start = start_position
        chunk_1_end = chunk_1_start + position_to_skip
        chunk_2_start = chunk_1_end + amount_to_skip
        chunk_2_end = chunk_1_start + (samples_to_extract - position_to_skip)
        
        chunk_1 = wraparound_extract(samples, chunk_1_start, chunk_1_end)
        chunk_2 = wraparound_extract(samples, chunk_2_start, chunk_2_end)
        chunk = np.concatenate((chunk_1, chunk_2))
    
    # Otherwise get contiguous chunk
    else:
        chunk = wraparound_extract(samples, start_position, samples_to_extract) 
        
    
    return chunk
    

get_chunk(samples = samples, sample_rate = sr)

array([-2.4658695e-05,  4.3753503e-06,  5.9822269e-06, ...,
        6.4597469e-07, -5.6203603e-06,  2.1704989e-06], dtype=float32)

### TODO: tests for `get_chunk()`

# Cyclic shift

In [7]:
def cyclic_shift(array, split_point = None):
    '''
    Shift array cyclicly by a random amount
    
    Shift array cyclicly by a random amount. Equivalent to
    splitting array into two parts at a random element, then
    switching the order of the parts.
    
    Args: 
        array (np.array): 1D-array to be split
        split_point (float): float in (0, 1) describing
            where in array to split -- for testing purposes.
            For stochastic splitting, leave as None.
    
    Returns:
        shifted_array: shifted array
    '''
    
    assert(type(array) == np.ndarray)
    length = array.shape[0]
    
    # Stochastic split point, or split point by floor of split_point * length of array
    if not split_point: split_point = random.randint(0, length)
    else: split_point = int(split_point * length)
    
    return np.concatenate((array[split_point:], array[:split_point]))

In [8]:
# Test random splitting
random.seed(100)
npt.assert_array_equal(cyclic_shift(np.array((0, 1, 2, 3, 4, 5, 6, 7))), np.array([2, 3, 4, 5, 6, 7, 0, 1])))

In [9]:
# Test deterministic splitting
npt.assert_array_equal(cyclic_shift(np.array([0, 1, 2]), split_point=0.5), np.array([1, 2, 0]))

# Test deterministic splitting
npt.assert_array_equal(cyclic_shift(np.array([0, 1, 2, 3]), split_point=0.5), np.array([2, 3, 0, 1]))

# Divided-samples augmentations: time & freq

### Helper function to divide samples randomly `divide_samples()`

In [51]:
def divide_samples(
    samples,
    sample_rate,
    low_duration = 0.5,
    high_duration = 5
):
    '''
    Divide audio samples into random-sized segments
    
    Divide audio samples into random-sized segments
    between the desired durations. The number
    of segments is not deterministic.
    
    Args
        samples (np.ndarray): 1d array of samples
        sample_rate (int): sample rate of samples
        low_duration (float): minimum duration
            in seconds of any segment
        high_duration (float): maximum duration
            in seconds of any segment
    
    Returns
        segments, list of sample lists
    '''
    
    min_chunk = int(low_duration * sample_rate)
    max_chunk = int(high_duration * sample_rate)
    
    samples_to_take = samples.copy()
    
    segments = []
    
    while samples_to_take.shape[0]:
        seg_size = random.randint(min_chunk, max_chunk)
        segment, samples_to_take = np.split(samples_to_take, [seg_size])
        segments.append(segment)
    
    return segments
    

In [17]:
# Test chunk division at set amount
array0 = np.array([0, 0, 0])
array1 = np.array([1, 1, 1])
array2 = np.array([2])
all_arrays = (array0, array1, array2)
cat_arrays = np.concatenate(all_arrays)
results = divide_chunk(cat_arrays, sample_rate=1, low_duration=3, high_duration=3)
for idx, result in enumerate(results):
    npt.assert_array_equal(result, all_arrays[idx])
    
# Test random chunk division
random.seed(333)
# Predetermined results with random.seed(333)
predetermined = [np.array([0, 1, 2, 3, 4, 5, 6, 7]), np.array([8, 9])]
results = divide_chunk(np.array(range(10)), sample_rate=1, low_duration=0, high_duration=10)
for idx, result in enumerate(results):
    npt.assert_array_equal(result, predetermined[idx])

### Helper function to concatenate divisions: `combine_samples()`

In [54]:
def combine_samples(divided):
    '''
    Combine divided samples back into a single array
    '''
    
    return np.concatenate(divided)

In [58]:
# Test that divided samples can be recombined successfully
samples, sr = librosa.load('../tests/silence_10s.mp3')
divided = divide_chunk(samples, sample_rate=sr, low_duration=0.5, high_duration=4)
npt.assert_array_equal(combine_samples(divided), samples)

### Time stretch the divisions `time_stretch_divisions()`

In [41]:
def time_stretch_divisions(
    divisions,
    chance_per_division = 0.50,
    mean_stretch = 1,
    sd_stretch = 0.05
):
    '''
    Time stretch divisions
    
    Given a list of np.ndarrays, each np.ndarray representing
    audio samples, stretch each array with some probability. 
    
    Args
        divisions (list of np.ndarrays) - list of np.ndarrays
            where each element of the list is samples from
            an audio file. A list of divisions can be generated 
            with helper functions in this module
        chance_per_division (float between 0 and 1) - for
            each division, the chance it will be time-stretched
        mean_stretch (float) - the mean stretch multiplier.
            == 1 is no stretch; > 1 is sped up, < 1 is slowed down
        sd_stretch (float > 0) - the sd of the stretch 
            distribution. 
    
    Returns
        stretched_divisions, time-stretched divisions
    '''
    stretched_divisions = []
    
    for d in divisions:
        if random.random() < chance_per_division:
            stretch_factor = np.random.normal(
                loc = mean_stretch,
                scale = sd_stretch)
            stretched_d = librosa.effects.time_stretch(y = d, rate = stretch_factor)
            stretched_divisions.append(stretched_d)
        else:
            stretched_divisions.append(d)
    
    return stretched_divisions

In [43]:
divs = divide_samples(
    samples = samples,
    sample_rate = sr,
    low_duration = 0.5,
    high_duration = 4)
print(divs)
print()
print(time_stretch_divisions(divs))

[array([-3.1739323e-06, -1.4155360e-05, -8.0013604e-07, ...,
       -1.1481586e-05,  6.8212121e-06,  1.0021004e-05], dtype=float32), array([-3.8107450e-05,  3.2519703e-05, -7.2488378e-06, ...,
       -4.7965218e-06, -5.5835485e-06,  1.4950513e-05], dtype=float32), array([-1.8977646e-05,  1.4777322e-05, -9.8069831e-06, ...,
       -6.1632509e-06,  9.4395991e-06, -5.3064232e-06], dtype=float32), array([-1.9714490e-07, -1.1566351e-05,  1.6784650e-06, ...,
        2.3299851e-06, -1.1209999e-06, -1.9890509e-05], dtype=float32)]

[array([-3.1739323e-06, -1.4155360e-05, -8.0013604e-07, ...,
       -1.1481586e-05,  6.8212121e-06,  1.0021004e-05], dtype=float32), array([-3.7972175e-05,  3.2426495e-05, -7.2842754e-06, ...,
       -4.0658415e-06,  4.0835298e-06,  3.5917142e-06], dtype=float32), array([-1.9036903e-05,  1.4823869e-05, -9.8147520e-06, ...,
       -3.2932824e-06,  6.1536948e-06,  1.0869603e-07], dtype=float32), array([-1.9714490e-07, -1.1566351e-05,  1.6784650e-06, ...,
        2.329

### Frequency shift the divisions `pitch_shift_divisions()`

In [52]:
def pitch_shift_divisions(
    divisions,
    sample_rate,
    chance_per_division = 0.40,
    mean_shift = 0,
    sd_shift = 0.25
):
    '''
    Time stretch divisions
    
    Given a list of np.ndarrays, each np.ndarray representing
    audio samples, pitch-shift each array with some probability. 
    The mean_shift and sd_shift should be given in "fractional
    half-steps," e.g. 0.25 = 1/4th of a half-step = 25 cents.
    
    Args
        divisions (list of np.ndarrays) - list of np.ndarrays
            where each element of the list is samples from
            an audio file. A list of divisions can be generated 
            with helper functions in this module
        sample_rate (int) - sample rate of all divisions
        chance_per_division (float between 0 and 1) - for
            each division, the chance it will be time-stretched
        mean_shift (float) - the mean pitch shift in (fractional) half-steps
            == 0 is no shift; > 0 is shift up; < 1 is shift down
        sd_shift (float > 0) - the sd of the shift 
            distribution in cents
    
    Returns
        shifted_divisions, pitch-shifted divisions
    '''
    shifted_divisions = []
    
    for d in divisions:
        if random.random() < chance_per_division:
            shift_factor = np.random.normal(
                loc = mean_shift,
                scale = sd_shift)
            shifted_d = librosa.effects.pitch_shift(
                y = d,
                sr = sample_rate,
                n_steps = shift_factor)
            shifted_divisions.append(shifted_d)
        else:
            shifted_divisions.append(d)
    
    return shifted_divisions

In [53]:
divs = divide_samples(
    samples = samples,
    sample_rate = sr,
    low_duration = 0.5,
    high_duration = 4)
print(divs)
print()
print(pitch_shift_divisions(divs, sample_rate = sr))

[array([-3.1739323e-06, -1.4155360e-05, -8.0013604e-07, ...,
        6.8143886e-06,  3.1100448e-05, -1.8384324e-05], dtype=float32), array([ 3.9387855e-06,  5.3881701e-07,  2.3175693e-05, ...,
        2.1746659e-05, -2.4835585e-05, -1.5028519e-06], dtype=float32), array([ 1.3826174e-05, -1.4743614e-05, -2.8227121e-06, ...,
        1.1666191e-06, -6.9062385e-06,  1.3662212e-05], dtype=float32), array([-1.76084395e-05,  1.62241304e-06,  1.51464765e-05, ...,
        2.32998514e-06, -1.12099985e-06, -1.98905091e-05], dtype=float32)]

[array([-3.1739323e-06, -1.4155360e-05, -8.0013604e-07, ...,
        6.8143886e-06,  3.1100448e-05, -1.8384324e-05], dtype=float32), array([ 2.9299094e-06,  8.2540856e-07,  2.2824666e-05, ...,
        1.2406320e-05, -4.7489721e-06,  0.0000000e+00], dtype=float32), array([ 1.3826174e-05, -1.4743614e-05, -2.8227121e-06, ...,
        1.1666191e-06, -6.9062385e-06,  1.3662212e-05], dtype=float32), array([-1.6713246e-05,  6.2172056e-08,  1.5658659e-05, ...,
       